## Multiple Traveling Salesman and the Problem of routing vehicles

Imagine we have instead of one salesman traveling to all the sites, that instead the workload is shared among many salesman. This generalization of the traveling salesman problem is called the multiple traveling salesman problem or mTSP. In lots of literature it is studied under the name of Vehicle Routing Problem or [VRP](https://en.wikipedia.org/wiki/Vehicle_routing_problem), but it is equivalent. 

In [1]:
from pulp import *
import numpy as np

### 1. First lets make some fake data

In [2]:
#a handful of sites
sites = ['org','A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T']

In [3]:
#non symetric distances
distances = dict( ((a,b),np.random.randint(10,50)) for a in sites for b in sites if a!=b )

### 2. The model

With a few modifications, the original traveling salesman problem can support multiple salesman. Instead of making each facility only be visited once, the origin facility will be visited multiple times. If we have two salesman then the origin is visited exactly twice and so on.

For $K$ vehicles or sales people

#### Variables:

indicators:
$$x_{i,j} = \begin{cases}
    1, & \text{if site i comes exactly before j in the tour} \\
    0, & \text{otherwise}
\end{cases}
$$

order dummy variables:
$$u_{i} :  \text{order site i is visited}$$


#### Minimize: 
$$\sum_{i,j \space i \neq j} x_{i,j} Distance(i,j)$$

#### Subject to:

$$\sum_{i \neq j} x_{i,j} = 1 \space \forall j \text{ except the origin}$$
$$\sum_{i \neq j} x_{i,origin} = K$$

$$\sum_{j \neq i} x_{i,j} = 1 \space \forall i \text{ except the origin}$$
$$\sum_{j \neq i} x_{i,origin} = K$$

$$u_{i}-u_{j} \leq  (N \div M)(1-x_{i,j}) - 1 \ \forall i,j \text{ except origins}$$

In [4]:
K = 5 #the number of sales people 

In [5]:
#create the problme
prob=LpProblem("salesman",LpMinimize)

In [6]:
#indicator variable if site i is connected to site j in the tour
x = LpVariable.dicts('x',distances, 0,1,LpBinary)
#dummy vars to eliminate subtours
u = LpVariable.dicts('u', sites, 0, len(sites)-1, LpInteger)

In [7]:
#the objective
cost = lpSum([x[(i,j)]*distances[(i,j)] for (i,j) in distances])
prob+=cost

In [8]:
#constraints
for k in sites:
    cap = 1 if k != 'org' else K
    #inbound connection
    prob+= lpSum([ x[(i,k)] for i in sites if (i,k) in x]) ==cap
    #outbound connection
    prob+=lpSum([ x[(k,i)] for i in sites if (k,i) in x]) ==cap
    
#subtour elimination
N=len(sites)/K
for i in sites:
    for j in sites:
        if i != j and (i != 'org' and j!= 'org') and (i,j) in x:
            prob += u[i] - u[j] <= (N)*(1-x[(i,j)]) - 1

### Solve it!

In [9]:
%time prob.solve()
print(LpStatus[prob.status])

CPU times: user 29.5 ms, sys: 6.77 ms, total: 36.3 ms
Wall time: 5.42 s
Optimal


And the result:

In [10]:
non_zero_edges = [ e for e in x if value(x[e]) != 0 ]

def get_next_site(parent):
    '''helper function to get the next edge'''
    edges = [e for e in non_zero_edges if e[0]==parent]
    for e in edges:
        non_zero_edges.remove(e)
    return edges

In [11]:
tours = get_next_site('org')
tours = [ [e] for e in tours ]

for t in tours:
    while t[-1][1] !='org':
        t.append(get_next_site(t[-1][1])[-1])

The optimal tours:

In [12]:
for t in tours:
    print(' -> '.join([ a for a,b in t]+['org']))

org -> M -> Q -> K -> E -> org
org -> R -> I -> P -> G -> org
org -> T -> H -> C -> S -> org
org -> O -> D -> N -> L -> org
org -> F -> A -> B -> J -> org


In [13]:
print(value(prob.objective))

358.0


### Questions:
1. If we wanted the vehicles/salesman to start and end in different locations, how would we modify this?
2. How can we limit the miles/km each vehicle/salesman drives?
3. How can we incorporate some sort of site priority?